In [158]:
# Load data

import pandas as pd

# Read CSV into a DataFrame
df = pd.read_csv("/Users/berni/Desktop/trial1.csv")

# Display first 5 rows
# print(df.head())

# Access specific column
# print(df['title'])

In [159]:
# Filter out uncredible words

# Define keywords to filter out
keywords = ['blog',
            'opinion',
            'click',
            'sponsored',
            'advertisement',
            'wordpress',
           'wix',
           'quora',
           'reddit',
           'weebly',
           'blogspot',
           'substack',
           'linkedin',
           'wikipedia',
           'are you',
           'job',
           'career',
           'shop',
           'sign in',
           '@',
           'youtube',
           'instagram']

mask = ~df.apply(
    lambda row: row.astype(str).str.contains(
        '|'.join(keywords),  # Regex "OR" pattern
        case=False)).any(axis=1)

# Apply the mask
df1 = df[mask]

print(df1['title'])

# Save the cleaned dataframe
# df.to_csv('cleaned_file.csv', index=False)

0                               Climate Data Online (CDO)
1                      Search | Climate Data Online (CDO)
2                          Maps & Data | NOAA Climate.gov
3                      Climate - National Weather Service
4                                               WorldClim
                              ...                        
2174    Adapting to the business future - Otago Daily ...
2175    NC Businesses Call for Commitment to Carbon Re...
2176    Aboitiz Foundation Champions Sustainability At...
2181    The Second Trump Administration and Europe's G...
2182    Op-Ed: Harrell's Growth Plan Shorts Housing an...
Name: title, Length: 2017, dtype: object


In [160]:
# Remove rows with links as titles

df1 = df1[df1['title'].str.split().str.len() != 1]

print(df1['link'])

0                      https://www.ncei.noaa.gov/cdo-web/
1                https://www.ncdc.noaa.gov/cdo-web/search
2                       https://www.climate.gov/maps-data
3                     https://www.weather.gov/wrh/climate
5           https://climateknowledgeportal.worldbank.org/
                              ...                        
2174    https://www.odt.co.nz/lifestyle/magazine/adapt...
2175    https://www.nrdc.org/press-releases/businesses...
2176    https://journal.com.ph/aboitiz-foundation-cham...
2181    https://www.globsec.org/what-we-do/commentarie...
2182    https://www.theurbanist.org/2025/03/29/harrell...
Name: link, Length: 1993, dtype: object


In [161]:
from geotext import GeoText

def extract_row_locations(row):
    """Process all text columns in a single row"""
    result = {}
    for col in ['title', 'link', 'snippet']:
        places = GeoText(str(row[col]))
        result[f'{col}_cities'] = places.cities
        result[f'{col}_countries'] = places.countries
    return pd.Series(result)

# Apply row-wise (axis=1) to maintain original length
location_df = df1.apply(extract_row_locations, axis=1)
df2 = pd.concat([df1, location_df], axis=1)

# Verify lengths match
assert len(df2) == len(df1)

print(df2)

                        query  rank  \
0            climate database     1   
1            climate database     2   
2            climate database     3   
3            climate database     4   
5            climate database     6   
...                       ...   ...   
2174  climate adaptation plan    90   
2175  climate adaptation plan    92   
2176  climate adaptation plan    93   
2181  climate adaptation plan    72   
2182  climate adaptation plan    80   

                                                  title  \
0                             Climate Data Online (CDO)   
1                    Search | Climate Data Online (CDO)   
2                        Maps & Data | NOAA Climate.gov   
3                    Climate - National Weather Service   
5                 Climate Change Knowledge Portal: Home   
...                                                 ...   
2174  Adapting to the business future - Otago Daily ...   
2175  NC Businesses Call for Commitment to Carbon Re...   


In [162]:
# Sort country names

import numpy as np
from geonamescache import GeonamesCache

loc_cols = df2.columns[-6:].tolist()  # Last 6 columns

for col in loc_cols:
    # Convert to string (handles lists safely) while keeping original index/rows
    df2[col] = df2[col].astype(str)

# Initialize and get all cities data
gc = GeonamesCache()
cities = gc.get_cities()

# Create city→country mapping (takes a few seconds to build)
city_to_country = {}
for city_id, city_data in cities.items():
    city_name = city_data['name'].lower()
    country_code = city_data['countrycode']
    country_name = gc.get_countries()[country_code]['name']
    city_to_country[city_name] = country_name

# 2. Process DataFrame columns
def extract_country(text):
    """Extract first valid city-country match from text"""
    if pd.isna(text):
        return None
    
    cities_found = GeoText(str(text)).cities
    for city in cities_found:
        if city.lower() in city_to_country:
            return city_to_country[city.lower()]
    return None

# 3. Apply to multiple columns in your DataFrame
def add_country_column(df2, loc_cols, new_col_name='city_to_country'):
    """
    Adds country column based on city detection in source columns
    Args:
        df: Input DataFrame
        source_cols: List of columns to search for cities
        new_col_name: Name for new country column
    """
    # Combine text from all source columns
    combined_text = df2[loc_cols].apply(
        lambda row: ' '.join(row.dropna().astype(str)), 
        axis=1
    )
    
    # Create new country column
    df2[new_col_name] = combined_text.apply(extract_country)
    return df2

# Example usage:
# Assuming df2 is your DataFrame and you want to process last 6 columns
df_loc = add_country_column(df2, loc_cols)


# Get unique values
unique_loc = df2['city_to_country'].astype(str).replace('nan', np.nan).dropna().unique()

# Count how many rows contain each substring
loc_counts = {word: df2['city_to_country'].str.contains(word, case=False).sum() 
          for word in unique_loc}
print(loc_counts)

# print(ft.columns)

{'None': 0, 'Philippines': 5, 'United States': 91, 'Turkey': 4, 'Uruguay': 1, 'France': 2, 'Argentina': 3, 'United Kingdom': 12, 'Netherlands': 2, 'Czechia': 3, 'Germany': 3, 'South Africa': 3, 'India': 1, 'Italy': 1, 'Kenya': 1, 'Canada': 3, 'Spain': 1, 'Chile': 1, 'Brazil': 1}


In [163]:
ft = df2.drop(['link_countries', 'link_cities'], axis=1)

import pandas as pd
from geotext import GeoText

def extract_countries(text):
    """Extract unique country names from text using GeoText"""
    if pd.isna(text):
        return []
    text = str(text)
    # Handle list-like strings if present
    if text.startswith('[') and text.endswith(']'):
        text = text[1:-1]  # Remove brackets
    countries = GeoText(text).countries
    return list(set(countries))  # Remove duplicates

# Specify columns to scan (last 5 columns in this case)
columns_to_scan = ['title_countries',
              'title_cities',
             'snippet_countries',
             'snippet_cities',
             'city_to_country']

columns_to_scan = [col for col in columns_to_scan if col in ft.columns]

# Create new column with aggregated country lists
ft['detected_countries'] = (
    ft[columns_to_scan]
    .apply(lambda row: list(set(
        country for col in row 
        for country in extract_countries(col)
    )), axis=1)
)

ft['countries'] = ft['detected_countries'].apply(
    lambda x: ', '.join(x) if x and len(x) > 0 else None
)

ft = ft.drop(['title_countries',
              'title_cities',
             'snippet_countries',
             'snippet_cities',
             'city_to_country'], axis=1)

print(ft)

                        query  rank  \
0            climate database     1   
1            climate database     2   
2            climate database     3   
3            climate database     4   
5            climate database     6   
...                       ...   ...   
2174  climate adaptation plan    90   
2175  climate adaptation plan    92   
2176  climate adaptation plan    93   
2181  climate adaptation plan    72   
2182  climate adaptation plan    80   

                                                  title  \
0                             Climate Data Online (CDO)   
1                    Search | Climate Data Online (CDO)   
2                        Maps & Data | NOAA Climate.gov   
3                    Climate - National Weather Service   
5                 Climate Change Knowledge Portal: Home   
...                                                 ...   
2174  Adapting to the business future - Otago Daily ...   
2175  NC Businesses Call for Commitment to Carbon Re...   


In [170]:
def extract_and_format_countries(text):
    """
    Extract countries using GeoText and format with commas
    Returns None if no countries found
    """
    if pd.isna(text) or not str(text).strip():
        return None
    
    # Extract unique countries (case-insensitive)
    countries = list(set(GeoText(str(text)).countries))
    
    if not countries:
        return None
    return ', '.join(countries) if len(countries) > 1 else countries[0]

# Create new formatted column
ft['countries'] = ft['countries'].apply(extract_and_format_countries)

print(ft.loc[6])

ft = ft.drop(['detected_countries'], axis=1)

ft.to_csv('TRIALFORCURSOR.csv', index=False)

query                                                  climate database
rank                                                                  7
title                                 Climate data for cities worldwide
link                                       https://en.climate-data.org/
snippet               In addition to weather and climate data for th...
page                                                                  1
detected_countries                          [Spain, Japan, Philippines]
countries                                     Spain, Japan, Philippines
Name: 6, dtype: object
